In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("data.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   crim     506 non-null    float64
 1   zn       506 non-null    float64
 2   indus    506 non-null    float64
 3   chas     506 non-null    int64  
 4   nox      506 non-null    float64
 5   rm       506 non-null    float64
 6   age      506 non-null    float64
 7   dis      506 non-null    float64
 8   rad      506 non-null    int64  
 9   tax      506 non-null    int64  
 10  ptratio  506 non-null    float64
 11  b        506 non-null    float64
 12  lstat    506 non-null    float64
 13  medv     506 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB


In [3]:
dataSet=data.drop("medv",axis=1)
dataLabels=data["medv"].copy()

In [4]:
from sklearn.model_selection import train_test_split
trainSet,testSet=train_test_split(dataSet,test_size=0.2,random_state=42)
trainLabels,testLabels=train_test_split(dataLabels,test_size=0.2,random_state=42)
print(f"Rows in trainSet: {len(trainSet)}\nRows in testSet: {len(testSet)}\n")
print(f"Rows in trainLabels: {len(trainLabels)}\nRows in testLabels: {len(testLabels)}\n")

Rows in trainSet: 404
Rows in testSet: 102

Rows in trainLabels: 404
Rows in testLabels: 102



In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

myPipeline=Pipeline([('imputer',SimpleImputer(strategy="median")),('stdScaler',StandardScaler()),])
finalTrainSet=myPipeline.fit_transform(trainSet)
finalTestSet=myPipeline.fit_transform(testSet)


In [6]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import LinearRegression
# from sklearn.tree import DecisionTreeRegressor

# model=LinearRegression()
# model=DecisionTreeRegressor()

model=RandomForestRegressor()
model.fit(finalTrainSet,trainLabels)
pred=model.predict(finalTrainSet)
mse=mean_squared_error(trainLabels,pred)
rmse=np.sqrt(mse)
mse     #overfitted in case of DecisionTree

1.8853799430693068

In [7]:
# To overcome overfitting and to evaluate models and choose the best one
from sklearn.model_selection import cross_val_score
scores=cross_val_score(model,finalTrainSet,trainLabels,scoring="neg_mean_squared_error",cv=10)
rmseScores=np.sqrt(-scores)  #scores will be in negative so to take sqrt add another minus
# rmseScores

In [8]:
def printScores(scores):
    print("Scores:", scores)
    print("Mean:",scores.mean())
    print("Standard Deviation:",scores.std())

printScores(rmseScores)

Scores: [2.99480604 3.48615593 2.53353579 5.4339488  4.82031172 2.88320047
 3.60917388 2.48742799 2.50060362 4.49472571]
Mean: 3.5243889951936453
Standard Deviation: 1.0036350665647402


In [9]:
# Creating the final model to be used
from joblib import dump
dump(model,'HousePricePredictor.joblib')

['HousePricePredictor.joblib']

### Testing against Test Data


In [10]:
finalPred=model.predict(finalTestSet)
finalMSE=mean_squared_error(testLabels,finalPred)
finalRMSE=np.sqrt(finalMSE)
# print(finalPred,list(testLabels))
finalRMSE

3.1987189868543853